# Setup environment

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

from PIL import Image
import numpy as np
import os

## Enable GPU

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

GPU device not found
Found GPU at: 


## Declare parameters

In [3]:
ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
MODEL_PATH = os.path.join(ROOT_DIR, "models", "binary_classification", "inception_v3", "transfer_learning.h5")

INPUT_SIZE = 299
BATCH_SIZE = 64

# Input test data

## Download 'non-food' dataset

I will use a downsampled ImageNet dataset for the 'non-food' images since it's free to access and has the broadest variety of images. However the images are limited to a resolution of 64x64 and they aren't labelled, so the 51 food classes will be treated as 'non-food'.

In [ ]:
non_food_test_raw = tfds.load("downsampled_imagenet/64x64", split="validation")

## Download 'food' dataset

I will the Food-101 dataset for the 'food' images, manually splitting with a 8:2 ratio of training to validation images.

In [ ]:
food_test_raw = tfds.load("food101", split="train[80:]")

## Format and label the images

Next, I will format the images for input into a convolutional neural network by rescaling the values between 0 & 1 and resizing to model input size. I will also label the data by labelling food images 1 and non-food images 0.

In [ ]:
def formatImage(img):
  img = tf.cast(img, tf.float32)
  img /= 255
  img = tf.image.resize_with_pad(img, INPUT_SIZE, INPUT_SIZE)
  return img

def formatNonFood(features):
  img = features["image"]
  img = formatImage(img)
  return img, 0

def formatFood(features):
  img = features["image"]
  img = formatImage(img)
  return img, 1


food_test_raw = food_test_raw.map(formatFood)
non_food_test_raw = non_test_val_raw.map(formatNonFood)

## Merge datasets

Concatenate the food and 'non-food' datsets.

In [ ]:
test_raw = food_test_raw.concatenate(non_food_test_raw)

## Prepare dataset

Shuffle and batch the images ready for training on the model.

In [ ]:
test_batches = test_raw.batch(BATCH_SIZE, drop_remainder=True) 

# Input model

In [4]:
model = tf.keras.models.load_model(MODEL_PATH, custom_objects={'KerasLayer':hub.KerasLayer})
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  21802784  
_________________________________________________________________
dense (Dense)                multiple                  2049      
Total params: 21,804,833
Trainable params: 2,049
Non-trainable params: 21,802,784
_________________________________________________________________


# Evaluate model on testing dataset

In [ ]:
loss, acc == model.evaluate(test_batches)

acc